## The following securities are included in the Markowitz Portfolio Optimization:
#### XLK, XLF, XLV, XLY, XLP, XLE, XLU, XLI, VNQ

In [1]:
etfs = ['XLK','XLF', 'XLV', 'XLY', 'XLP', 'XLE', 'XLU', 'XLI', 'VNQ']

## Collect security data from yfinance for a period of **10 Years**
##### *this is only done if data is not already pickled*
> You may refresh the data by deleting the security_data.pkl file

In [2]:
import os
import pickle
import yfinance as yf
security_data = None
if 'security_data.pkl' not in os.listdir():
    security_data = {etf: yf.Ticker(etf).history(period='10y') for etf in etfs}
    pickle.dump(security_data, open('security_data.pkl', 'wb+'))
else:
    security_data = pickle.load(open('security_data.pkl', 'rb'))

## Calculate returns and covariance matrix
>$R_t = \frac{p_t}{p_{t-1}} - 1$ where $p_t$ is the ***close*** price of a security at time t.

In [15]:
import pandas as pd
import numpy as np
return_data = {}
for etf in security_data:
    price_stream = security_data[etf]['Close']
    return_stream = [0]
    for i in range(1, len(price_stream)):
        return_stream.append(price_stream.iloc[i]/price_stream.iloc[i-1] - 1)
    return_data[etf] = pd.DataFrame({'Date': security_data[etf].index, 'Returns': return_stream})

returns_df = pd.DataFrame({etf: return_data[etf]['Returns'] for etf in return_data}) # returns dataframe where each security is a column vector;
covariance_df = returns_df.cov() # covariance matrix! in dataframe format
covariance_matrix = np.asmatrix(covariance_df)

In [16]:
covariance_matrix

matrix([[2.15176453e-04, 1.38668256e-04, 1.09086021e-04, 1.65280817e-04,
         8.15964433e-05, 1.25803395e-04, 7.63279085e-05, 1.34569220e-04,
         1.14963567e-04],
        [1.38668256e-04, 1.95373445e-04, 1.01853730e-04, 1.35829650e-04,
         8.06378900e-05, 1.80558477e-04, 8.14172341e-05, 1.51486995e-04,
         1.21109172e-04],
        [1.09086021e-04, 1.01853730e-04, 1.11894375e-04, 9.31873494e-05,
         6.96335932e-05, 9.41674412e-05, 7.19091026e-05, 9.44674976e-05,
         8.80494607e-05],
        [1.65280817e-04, 1.35829650e-04, 9.31873494e-05, 1.77460581e-04,
         7.43623433e-05, 1.23274191e-04, 7.11083848e-05, 1.28472750e-04,
         1.15291522e-04],
        [8.15964433e-05, 8.06378900e-05, 6.96335932e-05, 7.43623433e-05,
         8.53149651e-05, 7.25587538e-05, 7.95513022e-05, 7.71501425e-05,
         8.26156650e-05],
        [1.25803395e-04, 1.80558477e-04, 9.41674412e-05, 1.23274191e-04,
         7.25587538e-05, 3.55563777e-04, 8.06558442e-05, 1.59837688

In [92]:
def backtest(returns_dict, weights_dict):
    adjusted_returns_dict = {etf: returns_dict[etf].copy() for etf in returns_dict} # copy original returns dictionary
    for etf in adjusted_returns_dict: # adjusted returns simply add 1 to the 'Returns' so that you can multiple all returns together to get the total returns
        adjusted_returns_dict[etf]['Adjusted Returns'] = adjusted_returns_dict[etf]['Returns'] + 1
    total_returns = 0
    for etf in adjusted_returns_dict:
        total_returns += weights_dict[etf] * adjusted_returns_dict[etf]['Adjusted Returns'].cumprod().iloc[-1]
    return total_returns

def negated_backtest_1_arg(weights_dict):
    return -1 * backtest(return_data, weights_dict)

def weights_constraint(x):
    return numpy.asmatrix(x) @ np.ones(len(x),1) - 1

def variance_constraint(x):
    return numpy.asmatrix(x).T @ covariance_matrix @ numpy.asmatrix(x) - 0.05

In [93]:
weights_dict = {etf: 0 for etf in return_data}
weights_dict['XLE'] = 1
backtest(return_data, weights_dict)
negated_backtest_1_arg(weights_dict)

np.float64(-1.6538339702528866)

In [96]:
import scipy
from scipy.optimize import minimize

x0 = [1/len(return_data) for etf in return_data]
constraints = [{'type': 'eq', 'fun': weights_constraint}, {'type': 'ineq', 'fun': variance_constraint}]
bounds = [(0,1) for etf in return_data]
optimal_weights = scipy.optimize.minimize(negated_backtest_1_arg, x0, bounds=bounds, constraints=constraints)

ModuleNotFoundError: No module named 'scipy'